<a href="https://colab.research.google.com/github/LearnByDoing2024/Youtube/blob/main/groq_llama3_2_models_3B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
pip install pyautogen

In [ ]:
%%capture
pip install groq

In [ ]:
# @title
import os
from google.colab import userdata
from ipywidgets import widgets, VBox, Output
from IPython.display import display, clear_output
from pathlib import Path
import autogen
from autogen import AssistantAgent, UserProxyAgent
from autogen.coding import LocalCommandLineCodeExecutor

# Retrieve the Groq API Key from Google Secret using userdata
groq_api_key = userdata.get('GROQ_API')

if groq_api_key is None:
    raise ValueError("Groq API key not found. Please set 'GROQ_API' in Google Colab userdata.")

# Global config_list variable to be used by agents and the rest of the code
config_list = [
    {
        "model": "llama3-8b-8192",  # Default model
        "api_key": groq_api_key,
        "api_type": "groq",
    }
]

# Dropdown widget for model selection
model_dropdown = widgets.Dropdown(
    options=[
        ('Llama 3.2 1B Preview', 'llama-3.2-1b-preview'),
        ('Llama 3.2 3B Preview', 'llama-3.2-3b-preview'),
        ('Llama 3.2 11B Text Preview', 'llama-3.2-11b-text-preview'),
        ('Llama 3.2 90B Text Preview', 'llama-3.2-90b-text-preview'),
        ('Llama Guard 3 8B', 'llama-guard-3-8b'),
        ('LLaVA 1.5 7B', 'llava-v1.5-7b-4096-preview'),
        ('Meta Llama 3 70B', 'llama3-70b-8192'),
        ('Meta Llama 3 8B', 'llama3-8b-8192'),
        ('Mixtral 8x7B', 'mixtral-8x7b-32768'),
        ('Distil-Whisper English', 'distil-whisper-large-v3-en'),
        ('Gemma 2 9B', 'gemma2-9b-it'),
        ('Gemma 7B', 'gemma-7b-it'),
        ('Llama 3 Groq 70B Tool Use (Preview)', 'llama3-groq-70b-8192-tool-use-preview'),
        ('Llama 3 Groq 8B Tool Use (Preview)', 'llama3-groq-8b-8192-tool-use-preview'),
        ('Llama 3.1 405B (Offline)', 'llama-3.1-405b-offline'),
        ('Llama 3.1 70B Preview', 'llama-3.1-70b-versatile'),
        ('Llama 3.1 8B Preview', 'llama-3.1-8b-instant'),
        ('Whisper', 'whisper-large-v3')
    ],
    description='Model:',
    value='llama-3.2-1b-preview',  # Default selected value
)

# Button widget to trigger the execution
run_button = widgets.Button(
    description='Run',
    button_style='success'
)

# Output widget to display results
output = Output()

# Create the AssistantAgent and UserProxyAgent outside of the function
# Setting up the code executor
workdir = Path("coding")
workdir.mkdir(exist_ok=True)
code_executor = LocalCommandLineCodeExecutor(work_dir=workdir)

# Set up the UserProxyAgent
user_proxy_agent = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",  # Disable manual input and avoid EOFError
    code_execution_config={"executor": code_executor},
    is_termination_msg=lambda msg: "FINISH" in msg.get("content"),
)

# System message for the AssistantAgent
system_message = """You are a helpful AI assistant who writes code and the user executes it.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) for the user to execute.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
IMPORTANT: Wait for the user to execute your code and then you can reply with the word "FINISH". DO NOT OUTPUT "FINISH" after your code block."""

# Create the AssistantAgent with Groq configuration
assistant_agent = AssistantAgent(
    name="Groq Assistant",
    system_message=system_message,
    llm_config={"config_list": config_list},  # Use config list with Groq API
)

# Function to update the model and execute the chat
def run_model(b):
    with output:
        clear_output(wait=True)

    selected_model = model_dropdown.value

    # Update the global config_list with the selected model
    global config_list
    config_list = [
        {
            "model": selected_model,
            "api_key": groq_api_key,
            "api_type": "groq",
        }
    ]

    # Update the AssistantAgent with the new config
    assistant_agent.llm_config = {"config_list": config_list}

    # Start the chat, asking the AssistantAgent for code
    chat_result = user_proxy_agent.initiate_chat(
        assistant_agent,
        message="Provide code to count the number of prime numbers from 1 to 10000.",
    )

    # Display the result in the output widget
    display(chat_result)

# Attach the event to the button
run_button.on_click(run_model)

# Display the widgets
display(VBox([model_dropdown, run_button, output]))


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


User (to Groq Assistant):

Provide code to count the number of prime numbers from 1 to 10000.

--------------------------------------------------------------------------------
Groq Assistant (to User):

Here's a Python code snippet that will count the number of prime numbers from 1 to 10,000:

```python
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True

count = 0
for i in range(1, 10001):
    if is_prime(i):
        count += 1

print("Number of prime numbers from 1 to 10,000:", count)
```

In this code, we define a helper function `is_prime` that checks if a number is prime. We then use a for loop to iterate through numbers from 1 to 10,000, and increment a counter `count` whenever a prime number is found. Finally, we print the total count of prime numbers found.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK

ChatResult(chat_id=None, chat_history=[{'content': 'Provide code to count the number of prime numbers from 1 to 10000.', 'role': 'assistant', 'name': 'User'}, {'content': 'Here\'s a Python code snippet that will count the number of prime numbers from 1 to 10,000:\n\n```python\ndef is_prime(num):\n    if num < 2:\n        return False\n    for i in range(2, int(num**0.5) + 1):\n        if num % i == 0:\n            return False\n    return True\n\ncount = 0\nfor i in range(1, 10001):\n    if is_prime(i):\n        count += 1\n\nprint("Number of prime numbers from 1 to 10,000:", count)\n```\n\nIn this code, we define a helper function `is_prime` that checks if a number is prime. We then use a for loop to iterate through numbers from 1 to 10,000, and increment a counter `count` whenever a prime number is found. Finally, we print the total count of prime numbers found.', 'role': 'user', 'name': 'Groq Assistant'}, {'content': 'exitcode: 0 (execution succeeded)\nCode output: Number of prime n

In [ ]:
llm_config = {"config_list": config_list, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="TERMINATE",
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
user_proxy.initiate_chat(
    manager, message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software."
)
# type exit to terminate the chat

User_proxy (to chat_manager):

Find a latest paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------

Next speaker: Coder



/usr/local/lib/python3.10/dist-packages/autogen/oai/groq.py:286: UserWarning: Cost calculation not available for model llama-3.2-3b-preview
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Coder (to chat_manager):

# Step 1: Find the latest paper about GPT-4 on ArXiv
To find the latest paper about GPT-4 on ArXiv, we can use the arXiv grid search functionality.

```sh
grid search -- authors GPT-4 -- filed computer science -- filetype pdf
```

This command searches for PDF files containing the phrase "GPT-4" and the phrase "computer science," and returns a list of matching files.

# Step 2: Extract the title and authors of the latest paper
Assuming we obtained the list of papers from the previous step, we can extract the title and authors of the latest paper.

```sh
for d in /path/to/files/*; do 
 if [[ $d == *"GPT-4"* ]]; then
   title=$(grep title "$d" | cut -d'>' -f2 | cut -d "'" -f2)
   authors=$(grep authors "$d" | cut -d':' -f2 | tr -d ' ')
   echo "Title: $title, Authors: $authors"
 fi
done
```

# Step 3: Analyze the paper to identify potential applications in software
Assuming the latest paper was published by the authors mentioned, we can analyze the paper to iden

RuntimeError: Groq exception occurred: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.2-3b-preview` in organization `org_01hw1aerageqj8910pzkshh9xx` on tokens per minute (TPM): Limit 7000, Used 7075, Requested 7554. Please try again in 1m5.398857142s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}